In [1]:
from fastai.text import *
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import sentencepiece as spm
import re
import pdb

In [2]:
import fastai, torch
fastai.__version__ , torch.__version__

('1.0.57', '1.0.0')

In [3]:
torch.cuda.set_device(0)

In [4]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [5]:
random_seed(42, True)

In [6]:
path = Path('./')

In [7]:
df_train = pd.read_csv(path/'../../indicnlp-news-articles/kn/kn-train.csv', header=None)
df_train.head()

,0,1
0,sports,Samsung Galaxy M30s: ಲಭ್ಯವಾಗಲಿದೆ ಶಕ್ತಿಶಾಲಿ ಸ್ಮ...
1,sports,ಯುವೆಂಟಸ್ ತಂಡ ಸೇರಿದ ಕ್ರಿಸ್ಟಿಯಾನೊ ರೊನಾಲ್ಡೊ\nಯುವೆ...
2,sports,"ಬ್ಯಾಡ್ಮಿಂಟನ್‌: ಎರಡನೇ ಸುತ್ತಿಗೆ ಸಿಂಧು, ಸಮೀರ್‌\nಬ..."
3,entertainment,ತಮ್ಮದು ಒಪ್ಪಿತ ʼಸಂಬಂಧʼ ಎಂದು ಹೇಳಿದ ನಟ\n 12-05-2...
4,sports,ಖ್ಯಾತ ಕ್ರಿಕೆಟಿಗನ ಪತ್ನಿ ನಿಧನ\n 31-12-2018 7:57...


In [8]:
df_valid = pd.read_csv(path/'../../indicnlp-news-articles/kn/kn-valid.csv', header=None)
df_valid.head()

,0,1
0,sports,IPL 2018 ಟಾಸ್ ಗೆದ್ದ ಸಿಎಸ್’ಕೆ ಫೀಲ್ಡಿಂಗ್ ಆಯ್ಕೆ\n...
1,entertainment,‘ದೇಹಿ’ ಚಿತ್ರದಲ್ಲಿ ಕಳರಿಪಯಟ್ಟು ಆ್ಯಕ್ಷನ್‌\n‘ದೇಹಿ’...
2,entertainment,ಮಕ್ಕಳ ಬರ್ತಡೇಯನ್ನು ಸ್ಪೆಷಲ್ ಆಗಿ ಸೆಲಬ್ರೇಟ್ ಮಾಡಿದ ...
3,entertainment,ಕೆಂಪೇಗೌಡ 2ನಲ್ಲಿ ಸುದೀಪ್ ಇರುತ್ತಾರಾ?\nHighlights\...
4,sports,(ವಿಡಿಯೋ)ಜಾಧವ್ ಮೇಲೆ ಮೈದಾನದಲ್ಲೇ ಸಿಟ್ಟಾದ ಧೋನಿ!: ಆ...


In [9]:
df_test = pd.read_csv(path/'../../indicnlp-news-articles/kn/kn-test.csv', header=None)
df_test.head()

,0,1
0,sports,"ವಿಂಡೀಸ್‌ ತಂಡಕ್ಕೆ ಮರಳಿದ ಸುನೀಲ್‌, ಪೊಲ್ಲಾರ್ಡ್‌\n‌..."
1,entertainment,ರಂಗದಲ್ಲಿ ಕಮಲಾದೇವಿ ಚಟ್ಟೋಪಾಧ್ಯಾಯ\nkamaladevi cha...
2,entertainment,"Bengaluru, First Published 23, Nov 2018, 9:38 ..."
3,sports,"ಅಧ್ಯಯನ, ಅಭ್ಯಾಸದ ಗೊಂದಲದಲ್ಲಿ ಹಿಮಾ ದಾಸ್‌\nಅಧ್ಯಯನ,..."
4,lifestyle,ದೇಶ ಜೀವನದ ಕೊನೆಯ ಹಂತದಲ್ಲಿ ಆರೋಗ್ಯದ ದುಃಸ್ಥಿತಿ!\nJ...


In [10]:
df_train.shape, df_valid.shape, df_test.shape

((24000, 2), (3000, 2), (3000, 2))

In [11]:
df_train[df_train[0].isnull()].shape, df_valid[df_valid[0].isnull()].shape, df_test[df_test[0].isnull()].shape

((0, 2), (0, 2), (0, 2))

In [12]:
label_cols = [0]

In [13]:
class KannadaTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = lang
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str('./../../models/kannada/tokenizer/kannada_lm.model'))
        
    def tokenizer(self, t:str) -> List[str]:
        return self.sp.EncodeAsPieces(t)

In [14]:
sp = spm.SentencePieceProcessor()
sp.Load(str('./../../models/kannada/tokenizer/kannada_lm.model'))
itos = [sp.IdToPiece(int(i)) for i in range(25000)]

In [15]:
itos[:10]

['<unk>', '<s>', '</s>', '.', ',', '▁', '▁ಮತ್ತು', 'ದ', '▁ಈ', 'ಗಳು']

In [16]:
# 25,000 is the vocab size that we chose in sentencepiece
kannada_vocab = Vocab(itos)

In [17]:
tokenizer = Tokenizer(tok_func=KannadaTokenizer, lang='kn')

In [18]:
tokenizer.special_cases

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep']

In [19]:
data_lm = TextLMDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=kannada_vocab, label_cols=label_cols)

In [20]:
data_lm.show_batch()

idx,text
0,"▁ನೀಡ ುತ್ತಾಳೆ ▁ಎಂದು ▁ಕ್ರಿಕೆಟಿಗ ▁ಮಹೇಂದ್ರ ▁ಸಿಂಗ್ ▁ಧೋನಿ ▁ಹೇಳಿದ್ದಾರೆ . ▁ವಿಜಯ ▁ಕರ್ನಾಟಕ ▁| <unk> <unk> p d ated : <unk> <unk> un ▁12 , ▁2018 , ▁ 09 : 23 p m <unk> ▁i <unk> st ▁ಮುಂಬಯಿ : ▁ ಝಿ ವಾ ▁ನನ್ನನ್ನು ▁ಓರ್ವ ▁ಪರಿಪೂರ್ಣ ▁ವ್ಯಕ್ತಿಯಾಗಿ ▁ಬದಲಾಯಿಸಿದ ಳು . ▁ಹೆಣ್ಣುಮಕ್ಕಳು ▁ತಂದೆ ಗೆ ▁ಹೆಚ್ಚು ▁ನಿಕಟವಾಗಿ ರುತ್ತಾರೆ . ▁ಅದರಂತೆ ▁ ಝಿ ವಾ ▁ನನಗೆ ▁ಪ್ರತಿ ಬಾರಿ ಯೂ ▁ವಿಶೇಷ ▁ಆದ್ಯತೆ ▁ನೀಡ ುತ್ತಾಳೆ ▁ಎಂದು ▁ಕ್ರಿಕೆಟಿಗ ▁ಮಹೇಂದ್ರ ▁ಸಿಂಗ್"
1,ವನ್ನು ▁ಪಡೆಯಬಹುದು . ▁ಫ್ಲೂ ▁ಜ್ವರ ವನ್ನು ▁ ಕಡಿಮೆಗೊಳಿಸುತ್ತದೆ ▁ತುಳಸಿ ಯಲ್ಲಿರುವ ▁ಉರಿಯೂತ ▁ ನಿವಾರಕ ▁ಗುಣ ▁ಮತ್ತು ▁ಹಾಲಿನ ಲ್ಲಿರುವ ▁ಗಾಯ ವನ್ನು ▁ವಾಸಿಯಾಗ ಿಸುವ ▁ಗುಣಗಳು ▁ಮೇಳ ೈಸ ಿದಾಗ ▁ಫ್ಲೂ ▁ಜ್ವರ ದ ▁ವೈರಸ್ ▁ಗಳ ▁ಮೇಲೆ ▁ಹತೋಟಿ ▁ಸಾಧಿಸಲು ▁ಸಾಧ್ಯವಾಗುತ್ತದೆ . ▁ಒಂದೇ ▁ದಿನದಲ್ಲಿ ▁ಬಿಸಿ ಯಾಗಿದ್ದ ▁ದೇಹ ▁ಸಾಮಾನ್ಯ ▁ತಾಪಮಾನ ಕ್ಕಿಳಿಯ ುತ್ತದೆ . ▁ಹೃದಯದ ▁ಕ್ಷಮತೆ ▁ಹೆಚ್ಚಿಸುತ್ತದೆ ▁ತುಳಸಿ ಯಲ್ಲಿ ▁ಹಲವು ▁ಆಂಟಿ ▁ಆಕ್ಸಿಡ ೆಂಟು ಗಳಿದ್ದು ▁ಇವುಗಳಲ್ಲಿ ▁ಯೂ ಜಿ ನಾಲ್ ▁ <unk> e ug en ol ) ▁ಎಂಬುದು ▁ಹೃದಯ ವನ್ನು ▁ಬಲಪಡಿಸ
2,"▁ಇದ್ದ ▁15 ▁ಸ್ಪರ್ಧಿ ಗಳಲ್ಲಿ ▁ಈಗ ▁ಬಿಗ್ ▁ಮನೆಯಲ್ಲಿ ▁ಉಳಿದ ಿರುವುದು ▁ಕೇವಲ ▁7 ▁ಮಂದಿ . ▁ಇದೀಗ ▁ಈ ▁ಏಳು ▁ಮಂದಿ ಯ ▁ನಡುವೆ ▁ರಿಯಲ್ ▁ಫೈ ಟ್ ▁ಪ್ರಾರಂಭ ವಾಗಿದ್ದು , ▁ಎಲ್ಲರೂ ▁ತಾನು ▁ಗೆಲ್ಲ ಲೇ ಬೇಕೆಂಬ ▁ಹು ಮ್ಮ ಸ್ಸಿನ ಲ್ಲ ಿದ್ದಾರೆ . ▁ಬೆಂಗಳೂರು <unk> ಜ . 13 ) : ▁ಬಿಗ್ ▁ಬಾಸ್ ▁ಸೀ ಜನ್ ▁4 ▁ಮುಗಿಯ ಲು ▁ಇನ್ನು ▁ಕೆಲವೇ ▁ದಿನಗಳು ▁ಬಾಕಿ ▁ಉಳಿದಿವೆ . ▁ಶೋ ▁ಆರಂಭವಾದ ಾಗ ▁ಇದ್ದ ▁15 ▁ಸ್ಪರ್ಧಿ ಗಳಲ್ಲಿ ▁ಈಗ ▁ಬಿಗ್ ▁ಮನೆಯಲ್ಲಿ ▁ಉಳಿದ ಿರುವುದು ▁ಕೇವಲ ▁7"
3,"▁ಜೀವನದ ▁ಮೇಲೆ ▁ಪರಿಣಾಮ ▁ಬೀರುವ ುದಲ್ಲದೇ ▁ಅನೇಕ ▁ದೈಹಿಕ ▁ಹಾಗೂ ▁ಮಾನಸಿಕ ▁ಸಮಸ್ಯೆ ಗಳಿಗೂ ▁ಕಾರಣವಾಗುತ್ತವೆ . ▁ಆದರೆ ▁ಈ ▁ಒತ್ತಡ ▁ನಿವಾರಣೆ ಗೆ ▁ಆಯುರ್ವೇದ ▁ಪರಿಹಾರ ವಿದೆ . ▁ಉತ್ತಮ ▁ಆಹಾರ , ▁ವಿಚಾರ ▁ಹಾಗೂ ▁ಪಂಚ ಕರ್ಮ ▁ಚಿಕಿತ್ಸೆ ಗಳ ▁ಮೂಲಕ ▁ಒತ್ತಡವನ್ನು ▁ನಿವಾರಿಸಿ ಕೊಳ್ಳ ಬಹುದಾಗಿದೆ . ▁ಆಯುರ್ವೇದ ▁ಪರಿಹಾರ ▁ನಾವು ▁ರಜಾ ಕಾಲದಲ್ಲಿ ▁ಅರಣ್ಯ ಗಳಿಗೆ , ▁ಪರ್ವತ ಪ್ರದೇಶ ಗಳಿಗೆ , ▁ಸಮುದ್ರತೀರ ಗಳಿಗೆ ▁ಪ್ರವಾಸ ▁ಹೋಗಲು ▁ಬಯಸ ುತ್ತೇವೆ . ▁ಇದೆ ಲ್ಲಾ ▁ನಮ್ಮ ▁ದೈನಂದಿನ ▁ಬದುಕಿನ ▁ಒತ್ತಡ ದಿಂದ ▁ತಪ್ಪಿಸಿಕೊಳ್ಳುವ ▁ಉಪಾಯ ಗಳಾಗಿರುತ್ತವೆ . ▁ಈ ▁ರೀತಿಯ ▁ಪ್ರವಾಸ"
4,") , ▁ದಿವ್ಯ ಾ ▁ಸ ತೀ ಜಾ ▁ <unk> 1 : 01 . 61 ▁ಸೆ . ) , ▁ಶಿವ ಾನಿ ▁ಕಟಾ ರಿಯಾ ▁ <unk> 59 . 57 ▁ಸೆ . ) ▁ಮತ್ತು ▁ಮಾ ನಾ ▁ಪಟೇಲ್ ▁ <unk> 59 . 75 ▁ಸೆ . ) ▁ಅವರ ನ್ನೊಳಗೊಂಡ ▁ಭಾರತ ▁ಮಹಿಳಾ ▁ತಂಡ ▁4 : 00 . 76 ▁ಸೆಕೆಂಡ್ ▁ಗಳಲ್ಲಿ ▁4 x 100 ▁ರೇಸ್ ▁ಪೂರೈಸಿದರು . ▁ಇದೇ ▁ವಿಭಾಗದಲ್ಲಿ ▁ಥಾಯ್ ಲೆಂಡ್ ▁ <unk> 3 : 54 ."


In [21]:
awd_lstm_config = awd_lstm_lm_config.copy()
awd_lstm_config['n_hid'] = 1150
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.3, config=awd_lstm_config, pretrained=False)

In [22]:
# Loading the pretrained language model on kannada wikipedia
learn.load('../../../models/kannada/lm/ULMFiT/third_kn_lm', with_opt=True)

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (24000 items)
x: LMTextList
▁x x b os <unk> ms ung <unk> ▁g ala x y <unk> ▁m 30 s : ▁ಲಭ್ಯ ವಾಗಲಿದೆ ▁ಶಕ್ತಿಶಾಲಿ ▁ಸ್ಮಾರ್ಟ್ ▁ಫೋನ್ <unk> ▁w <unk> at <unk> ch <unk> ▁l <unk> iv e <unk> <unk> ▁ಮಗಳು ▁ನನ್ನನ್ನು ▁ಪರಿಪೂರ್ಣ ▁ವ್ಯಕ್ತಿಯಾಗಿ ▁ಬದಲಾಯಿಸಿದ ಳು ; ▁ಧೋನಿ ▁ ಝಿ ವಾ ▁ನನ್ನನ್ನು ▁ಓರ್ವ ▁ಪರಿಪೂರ್ಣ ▁ವ್ಯಕ್ತಿಯಾಗಿ ▁ಬದಲಾಯಿಸಿದ ಳು . ▁ಹೆಣ್ಣುಮಕ್ಕಳು ▁ತಂದೆ ಗೆ ▁ಹೆಚ್ಚು ▁ನಿಕಟವಾಗಿ ರುತ್ತಾರೆ . ▁ಅದರಂತೆ ▁ ಝಿ ವಾ ▁ನನಗೆ ▁ಪ್ರತಿ ಬಾರಿ ಯೂ ▁ವಿಶೇಷ ▁ಆದ್ಯತೆ ▁ನೀಡ ುತ್ತಾಳೆ ▁ಎಂದು ▁ಕ್ರಿಕೆಟಿಗ ▁ಮಹೇಂದ್ರ ▁ಸಿಂಗ್ ▁ಧೋನಿ ▁ಹೇಳಿದ್ದಾರೆ . ▁ವಿಜಯ ▁ಕರ್ನಾಟಕ ▁| <unk> <unk> p d ated : <unk> <unk> un ▁12 , ▁2018 , ▁ 09 : 23 p m <unk> ▁i <unk> st ▁ಮುಂಬಯಿ : ▁ ಝಿ ವಾ ▁ನನ್ನನ್ನು ▁ಓರ್ವ ▁ಪರಿಪೂರ್ಣ ▁ವ್ಯಕ್ತಿಯಾಗಿ ▁ಬದಲಾಯಿಸಿದ ಳು . ▁ಹೆಣ್ಣುಮಕ್ಕಳು ▁ತಂದೆ ಗೆ ▁ಹೆಚ್ಚು ▁ನಿಕಟವಾಗಿ ರುತ್ತಾರೆ . ▁ಅದರಂತೆ ▁ ಝಿ ವಾ ▁ನನಗೆ ▁ಪ್ರತಿ ಬಾರಿ ಯೂ ▁ವಿಶೇಷ ▁ಆದ್ಯತೆ ▁ನೀಡ ುತ್ತಾಳೆ ▁ಎಂದು ▁ಕ್ರಿಕೆಟಿಗ ▁ಮಹೇಂದ್ರ ▁ಸಿಂಗ್ ▁ಧೋನಿ ▁ಹೇಳಿದ್ದಾರೆ . ▁ಈ ▁ಬಾರಿ ಯ ▁ಐಪಿಎಲ್ ▁ಪಂದ್ಯಗಳಲ್ಲಿ ▁ ಝಿ ವಾ ▁ಧೋನಿ ▁ಜತೆ ▁ಕಾಣಿಸಿಕೊಳ್ಳ ುತ್ತಿದ್ದಳು . ▁ಕ್ರಿಕೆಟ್ ▁ನಿಂದಾಗಿ ▁ಬಹುತೇಕ ▁ಸಂದರ್ಭದಲ್ಲಿ ▁ಮ ಗ

In [23]:
learn.freeze()

In [24]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.600813,3.632022,0.405694,04:32


In [25]:
learn.unfreeze()

In [26]:
learn.fit_one_cycle(2, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.321611,3.389092,0.433200,05:19
1,3.319640,3.309385,0.442952,05:19


In [27]:
learn.predict('ರಾಮ್',n_words=10)

'ರಾಮ್ ▁ಚರಣ್ ▁ರೈ ▁2 ▁ಆಗಸ್ಟ್ ▁20 19 , ▁ 01 :'

In [28]:
learn.save_encoder('fine_tuned_enc')

In [29]:
data_clas = TextClasDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=kannada_vocab, label_cols=label_cols, bs=64)

In [30]:
data_clas.show_batch()

text,target
"▁x x b os ▁ಸುದೀರ್ಘ ▁ಬರಹ : ▁ ಸೂರಿ ▁ಹೇಳಿದ ▁ಆ ▁ಕ್ಷಣ ದ ▁ಸತ್ಯ ಗಳು ! ▁ಇ ವ್ ರು ▁ಧ ಮ್ ▁ಬಿ ರಿಯಾ ನಿ ▁ಸ್ಪೆ ಷ ಲಿಸ್ಟ್ ▁! ▁ಸುದೀರ್ಘ ▁ಬರಹ : ▁ ಸೂರಿ ▁ಹೇಳಿದ ▁ಆ ▁ಕ್ಷಣ ದ ▁ಸತ್ಯ ಗಳು ! ▁ರಘುನಾಥ ▁ಚ . ಹ . ▁– ▁ಪದ್ಮನಾಭ ▁ಭಟ್ <unk> ▁p ub l ish ed : ▁ 04 ▁ಸೆಪ್ಟೆಂಬರ್ ▁2018 , ▁13 : 45 <unk> ▁i <unk> st <unk> <unk> p",entertainment
▁x x b os ▁ಮಾಸ ▁ಮೇ ಷ ▁ಯಾವುದೇ ▁ವಿಚಾರದಲ್ಲಿ ▁ಆತ ು ರ ತೆಯ ▁ತೀರ್ಮಾನ ▁ಬೇಡ . ▁ಬಿಡುವಿಲ್ಲದ ▁ಕೆಲಸ ▁ಕಾರ್ಯ ಗಳಿಂದಾಗಿ ▁ಒತ್ತಡ ▁ಹೆಚ್ಚಾಗಿ ▁ಮಾನಸಿಕ ▁ಚಂಚಲ ತೆ ▁ಮತ್ತು ▁ಆಯಾಸ . ▁ಆರೋಗ್ಯದ ▁ಬಗ್ಗೆ ▁ನಿರ್ಲಕ್ಷ್ಯ ದಿಂದಾಗಿ ▁ಪ್ರತಿಕೂಲ ▁ಪರಿಣಾಮ ▁ಉಂಟಾಗಬಹುದು . ▁ವೃಷಭ ▁ಪ್ರಯಾಣ ▁ಕಾಲದಲ್ಲಿ ▁ಎಚ್ಚರಿಕೆ ▁ಅಗತ್ಯ . ▁ಧೈರ್ಯ ದಿಂದ ▁ಪ್ರಮುಖ ▁ನಿರ್ಧಾರಗಳನ್ನು ▁ಕೈಗೊಳ್ಳ ಬಹುದು . ▁ಕೆಲಸ ▁ಕಾರ್ಯಗಳಿಗೆ ▁ಕಾಲ ಮಿತಿ ▁ಹಾಕಿ ಕೊಳ್ಳುವುದು ▁ಉತ್ತಮ . ▁ಸಾಂ ಸಾರ ಿಕವಾಗಿ ▁ತೃಪ್ತಿ ಕರ ▁ಜೀವನ . ▁ಧಾರ್ಮಿಕ ▁ಕೆಲಸ ▁ಕಾರ್ಯಗಳಲ್ಲಿ ▁ಭಾಗವಹಿಸುವ ▁ಸಾಧ್ಯತೆ .,entertainment
▁x x b os ▁ಮಾಸ ▁ಮೇ ಷ ▁ಯಾವುದೇ ▁ವಿಚಾರದಲ್ಲಿ ▁ಆತ ು ರ ತೆಯ ▁ತೀರ್ಮಾನ ▁ಬೇಡ . ▁ಬಿಡುವಿಲ್ಲದ ▁ಕೆಲಸ ▁ಕಾರ್ಯ ಗಳಿಂದಾಗಿ ▁ಒತ್ತಡ ▁ಹೆಚ್ಚಾಗಿ ▁ಮಾನಸಿಕ ▁ಚಂಚಲ ತೆ ▁ಮತ್ತು ▁ಆಯಾಸ . ▁ಆರೋಗ್ಯದ ▁ಬಗ್ಗೆ ▁ನಿರ್ಲಕ್ಷ್ಯ ದಿಂದಾಗಿ ▁ಪ್ರತಿಕೂಲ ▁ಪರಿಣಾಮ ▁ಉಂಟಾಗಬಹುದು . ▁ವೃಷಭ ▁ಪ್ರಯಾಣ ▁ಕಾಲದಲ್ಲಿ ▁ಎಚ್ಚರಿಕೆ ▁ಅಗತ್ಯ . ▁ಧೈರ್ಯ ದಿಂದ ▁ಪ್ರಮುಖ ▁ನಿರ್ಧಾರಗಳನ್ನು ▁ಕೈಗೊಳ್ಳ ಬಹುದು . ▁ಕೆಲಸ ▁ಕಾರ್ಯಗಳಿಗೆ ▁ಕಾಲ ಮಿತಿ ▁ಹಾಕಿ ಕೊಳ್ಳುವುದು ▁ಉತ್ತಮ . ▁ಸಾಂ ಸಾರ ಿಕವಾಗಿ ▁ತೃಪ್ತಿ ಕರ ▁ಜೀವನ . ▁ಧಾರ್ಮಿಕ ▁ಕೆಲಸ ▁ಕಾರ್ಯಗಳಲ್ಲಿ ▁ಭಾಗವಹಿಸುವ ▁ಸಾಧ್ಯತೆ .,sports
"▁x x b os ▁ಮಾಸ ▁ಮೇ ಷ ▁ಕಳೆದುಹೋದ ▁ಅಥವಾ ▁ಬೇರೆಯವರ ▁ವಶ ದಲ್ಲಿದ್ದ ▁ನಿಮ್ಮ ▁ವಸ್ತುಗಳನ್ನು ▁ಪುನಃ ▁ಪಡೆದುಕೊಳ್ಳುವ ▁ಸಾಧ್ಯತೆ . ▁ವಾಹನ , ▁ಯಂತ್ರ ೋ ಕ ಪರ ಣ ▁ಆಸ್ತಿ ಗಳ ▁ಖರೀದಿ . ▁ದೇವತಾ ▁ಕಾರ್ಯಗಳಲ್ಲಿ ▁ತೊಡಗಿಕೊಂಡ ು ▁ನೆಮ್ಮದಿ . ▁ವೃಷಭ ▁ಸಂಬಂಧಿಕರ ▁ಜತೆ ▁ವಿಚಾರ ▁ವಿನಿಮಯ ▁ನಡೆಸಿ , ▁ಆರ್ಥಿಕ ▁ಸಂಕಷ್ಟ ▁ಪರಿಹರಿಸ ಿಕೊಳ್ಳುವ ಿರಿ . ▁ಓದಿ ನಲ್ಲಿ ▁ಆಸಕ್ತಿ ▁ತೋರುವ ಿರಿ . ▁ಹೊಸ ▁ಉದ್ಯಮ ಕ್ಕೆ ▁ತಯಾರಿ ▁ನಡೆಸ ಲ ಿದ್ದ ೀ ರಿ . ▁ಮನೆ ಮಂದಿ ಯೊಂದಿಗೆ ▁ಸಂತೋಷ ದಿಂದ ▁ಕಾಲ",sports
"▁x x b os ▁ಮಾಸ ▁ಮೇ ಷ ▁ಕಳೆದುಹೋದ ▁ಅಥವಾ ▁ಬೇರೆಯವರ ▁ವಶ ದಲ್ಲಿದ್ದ ▁ನಿಮ್ಮ ▁ವಸ್ತುಗಳನ್ನು ▁ಪುನಃ ▁ಪಡೆದುಕೊಳ್ಳುವ ▁ಸಾಧ್ಯತೆ . ▁ವಾಹನ , ▁ಯಂತ್ರ ೋ ಕ ಪರ ಣ ▁ಆಸ್ತಿ ಗಳ ▁ಖರೀದಿ . ▁ದೇವತಾ ▁ಕಾರ್ಯಗಳಲ್ಲಿ ▁ತೊಡಗಿಕೊಂಡ ು ▁ನೆಮ್ಮದಿ . ▁ವೃಷಭ ▁ಸಂಬಂಧಿಕರ ▁ಜತೆ ▁ವಿಚಾರ ▁ವಿನಿಮಯ ▁ನಡೆಸಿ , ▁ಆರ್ಥಿಕ ▁ಸಂಕಷ್ಟ ▁ಪರಿಹರಿಸ ಿಕೊಳ್ಳುವ ಿರಿ . ▁ಓದಿ ನಲ್ಲಿ ▁ಆಸಕ್ತಿ ▁ತೋರುವ ಿರಿ . ▁ಹೊಸ ▁ಉದ್ಯಮ ಕ್ಕೆ ▁ತಯಾರಿ ▁ನಡೆಸ ಲ ಿದ್ದ ೀ ರಿ . ▁ಮನೆ ಮಂದಿ ಯೊಂದಿಗೆ ▁ಸಂತೋಷ ದಿಂದ ▁ಕಾಲ",entertainment


In [31]:
del awd_lstm_config['tie_weights']
del awd_lstm_config['out_bias']

In [32]:
learn = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.5, config=awd_lstm_config)

In [33]:
learn.load_encoder('fine_tuned_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (24000 items)
x: TextList
▁x x b os <unk> ms ung <unk> ▁g ala x y <unk> ▁m 30 s : ▁ಲಭ್ಯ ವಾಗಲಿದೆ ▁ಶಕ್ತಿಶಾಲಿ ▁ಸ್ಮಾರ್ಟ್ ▁ಫೋನ್ <unk> ▁w <unk> at <unk> ch <unk> ▁l <unk> iv e <unk> <unk> ▁ಮಗಳು ▁ನನ್ನನ್ನು ▁ಪರಿಪೂರ್ಣ ▁ವ್ಯಕ್ತಿಯಾಗಿ ▁ಬದಲಾಯಿಸಿದ ಳು ; ▁ಧೋನಿ ▁ ಝಿ ವಾ ▁ನನ್ನನ್ನು ▁ಓರ್ವ ▁ಪರಿಪೂರ್ಣ ▁ವ್ಯಕ್ತಿಯಾಗಿ ▁ಬದಲಾಯಿಸಿದ ಳು . ▁ಹೆಣ್ಣುಮಕ್ಕಳು ▁ತಂದೆ ಗೆ ▁ಹೆಚ್ಚು ▁ನಿಕಟವಾಗಿ ರುತ್ತಾರೆ . ▁ಅದರಂತೆ ▁ ಝಿ ವಾ ▁ನನಗೆ ▁ಪ್ರತಿ ಬಾರಿ ಯೂ ▁ವಿಶೇಷ ▁ಆದ್ಯತೆ ▁ನೀಡ ುತ್ತಾಳೆ ▁ಎಂದು ▁ಕ್ರಿಕೆಟಿಗ ▁ಮಹೇಂದ್ರ ▁ಸಿಂಗ್ ▁ಧೋನಿ ▁ಹೇಳಿದ್ದಾರೆ . ▁ವಿಜಯ ▁ಕರ್ನಾಟಕ ▁| <unk> <unk> p d ated : <unk> <unk> un ▁12 , ▁2018 , ▁ 09 : 23 p m <unk> ▁i <unk> st ▁ಮುಂಬಯಿ : ▁ ಝಿ ವಾ ▁ನನ್ನನ್ನು ▁ಓರ್ವ ▁ಪರಿಪೂರ್ಣ ▁ವ್ಯಕ್ತಿಯಾಗಿ ▁ಬದಲಾಯಿಸಿದ ಳು . ▁ಹೆಣ್ಣುಮಕ್ಕಳು ▁ತಂದೆ ಗೆ ▁ಹೆಚ್ಚು ▁ನಿಕಟವಾಗಿ ರುತ್ತಾರೆ . ▁ಅದರಂತೆ ▁ ಝಿ ವಾ ▁ನನಗೆ ▁ಪ್ರತಿ ಬಾರಿ ಯೂ ▁ವಿಶೇಷ ▁ಆದ್ಯತೆ ▁ನೀಡ ುತ್ತಾಳೆ ▁ಎಂದು ▁ಕ್ರಿಕೆಟಿಗ ▁ಮಹೇಂದ್ರ ▁ಸಿಂಗ್ ▁ಧೋನಿ ▁ಹೇಳಿದ್ದಾರೆ . ▁ಈ ▁ಬಾರಿ ಯ ▁ಐಪಿಎಲ್ ▁ಪಂದ್ಯಗಳಲ್ಲಿ ▁ ಝಿ ವಾ ▁ಧೋನಿ ▁ಜತೆ ▁ಕಾಣಿಸಿಕೊಳ್ಳ ುತ್ತಿದ್ದಳು . ▁ಕ್ರಿಕೆಟ್ ▁ನಿಂದಾಗಿ ▁ಬಹುತೇಕ ▁ಸಂದರ್ಭದಲ್ಲಿ ▁ಮ ಗಳ ▁ಜತ

In [34]:
learn.freeze()

In [35]:
learn.loss_func.func

CrossEntropyLoss()

In [36]:
mcc = MatthewsCorreff()

In [37]:
learn.metrics = [mcc, accuracy]

In [38]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.107515,0.069774,0.965071,0.976667,01:45


In [39]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.053790,0.032763,0.979561,0.986333,02:03


In [40]:
learn.save('second-full')

In [41]:
learn.unfreeze()
learn.fit_one_cycle(5, 1e-3, callbacks=[callbacks.SaveModelCallback(learn, every='improvement', monitor='accuracy', name='final')])

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.038112,0.030786,0.979572,0.986333,04:02
1,0.031781,0.033764,0.978588,0.985667,04:02
2,0.019479,0.040259,0.980049,0.986667,04:02
3,0.020642,0.032843,0.981543,0.987667,04:02
4,0.006806,0.031738,0.982048,0.988000,04:02


Better model found at epoch 0 with accuracy value: 0.9863333106040955.
Better model found at epoch 2 with accuracy value: 0.9866666793823242.
Better model found at epoch 3 with accuracy value: 0.987666666507721.
Better model found at epoch 4 with accuracy value: 0.9879999756813049.


In [42]:
learn.load('final')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (24000 items)
x: TextList
▁x x b os <unk> ms ung <unk> ▁g ala x y <unk> ▁m 30 s : ▁ಲಭ್ಯ ವಾಗಲಿದೆ ▁ಶಕ್ತಿಶಾಲಿ ▁ಸ್ಮಾರ್ಟ್ ▁ಫೋನ್ <unk> ▁w <unk> at <unk> ch <unk> ▁l <unk> iv e <unk> <unk> ▁ಮಗಳು ▁ನನ್ನನ್ನು ▁ಪರಿಪೂರ್ಣ ▁ವ್ಯಕ್ತಿಯಾಗಿ ▁ಬದಲಾಯಿಸಿದ ಳು ; ▁ಧೋನಿ ▁ ಝಿ ವಾ ▁ನನ್ನನ್ನು ▁ಓರ್ವ ▁ಪರಿಪೂರ್ಣ ▁ವ್ಯಕ್ತಿಯಾಗಿ ▁ಬದಲಾಯಿಸಿದ ಳು . ▁ಹೆಣ್ಣುಮಕ್ಕಳು ▁ತಂದೆ ಗೆ ▁ಹೆಚ್ಚು ▁ನಿಕಟವಾಗಿ ರುತ್ತಾರೆ . ▁ಅದರಂತೆ ▁ ಝಿ ವಾ ▁ನನಗೆ ▁ಪ್ರತಿ ಬಾರಿ ಯೂ ▁ವಿಶೇಷ ▁ಆದ್ಯತೆ ▁ನೀಡ ುತ್ತಾಳೆ ▁ಎಂದು ▁ಕ್ರಿಕೆಟಿಗ ▁ಮಹೇಂದ್ರ ▁ಸಿಂಗ್ ▁ಧೋನಿ ▁ಹೇಳಿದ್ದಾರೆ . ▁ವಿಜಯ ▁ಕರ್ನಾಟಕ ▁| <unk> <unk> p d ated : <unk> <unk> un ▁12 , ▁2018 , ▁ 09 : 23 p m <unk> ▁i <unk> st ▁ಮುಂಬಯಿ : ▁ ಝಿ ವಾ ▁ನನ್ನನ್ನು ▁ಓರ್ವ ▁ಪರಿಪೂರ್ಣ ▁ವ್ಯಕ್ತಿಯಾಗಿ ▁ಬದಲಾಯಿಸಿದ ಳು . ▁ಹೆಣ್ಣುಮಕ್ಕಳು ▁ತಂದೆ ಗೆ ▁ಹೆಚ್ಚು ▁ನಿಕಟವಾಗಿ ರುತ್ತಾರೆ . ▁ಅದರಂತೆ ▁ ಝಿ ವಾ ▁ನನಗೆ ▁ಪ್ರತಿ ಬಾರಿ ಯೂ ▁ವಿಶೇಷ ▁ಆದ್ಯತೆ ▁ನೀಡ ುತ್ತಾಳೆ ▁ಎಂದು ▁ಕ್ರಿಕೆಟಿಗ ▁ಮಹೇಂದ್ರ ▁ಸಿಂಗ್ ▁ಧೋನಿ ▁ಹೇಳಿದ್ದಾರೆ . ▁ಈ ▁ಬಾರಿ ಯ ▁ಐಪಿಎಲ್ ▁ಪಂದ್ಯಗಳಲ್ಲಿ ▁ ಝಿ ವಾ ▁ಧೋನಿ ▁ಜತೆ ▁ಕಾಣಿಸಿಕೊಳ್ಳ ುತ್ತಿದ್ದಳು . ▁ಕ್ರಿಕೆಟ್ ▁ನಿಂದಾಗಿ ▁ಬಹುತೇಕ ▁ಸಂದರ್ಭದಲ್ಲಿ ▁ಮ ಗಳ ▁ಜತ

In [43]:
from sklearn.metrics import accuracy_score, matthews_corrcoef
df_dict = {'query': list(df_test[1]), 'actual_label': list(df_test[0]), 'predicted_label': ['']*df_test.shape[0]}
all_nodes = list(set(df_train[0]))
for node in all_nodes:
    df_dict[node] = ['']*df_test.shape[0]
    
i2c = {}
for key, value in learn.data.c2i.items():
    i2c[value] = key
    
df_result = pd.DataFrame(df_dict)
preds = learn.get_preds(ds_type=DatasetType.Test, ordered=True)
for index, row in df_result.iterrows():
    for node in all_nodes:
        row[node] = preds[0][index][learn.data.c2i[node]].item()
    row['predicted_label'] = i2c[np.argmax(preds[0][index]).data.item()]
df_result.head()

,query,actual_label,predicted_label,lifestyle,entertainment,sports
0,"ವಿಂಡೀಸ್‌ ತಂಡಕ್ಕೆ ಮರಳಿದ ಸುನೀಲ್‌, ಪೊಲ್ಲಾರ್ಡ್‌\n‌...",sports,sports,3.8855e-05,0.000298875,0.999662
1,ರಂಗದಲ್ಲಿ ಕಮಲಾದೇವಿ ಚಟ್ಟೋಪಾಧ್ಯಾಯ\nkamaladevi cha...,entertainment,entertainment,4.96969e-05,0.999891,5.90762e-05
2,"Bengaluru, First Published 23, Nov 2018, 9:38 ...",entertainment,entertainment,1.04912e-06,0.999996,2.78782e-06
3,"ಅಧ್ಯಯನ, ಅಭ್ಯಾಸದ ಗೊಂದಲದಲ್ಲಿ ಹಿಮಾ ದಾಸ್‌\nಅಧ್ಯಯನ,...",sports,sports,5.50104e-05,1.14871e-06,0.999944
4,ದೇಶ ಜೀವನದ ಕೊನೆಯ ಹಂತದಲ್ಲಿ ಆರೋಗ್ಯದ ದುಃಸ್ಥಿತಿ!\nJ...,lifestyle,lifestyle,1,1.08284e-07,1.50986e-07


In [44]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])

0.9886666666666667

In [45]:
matthews_corrcoef(df_result['actual_label'], df_result['predicted_label'])

0.9830150081641603

In [46]:
df_result.to_csv('indicnlp_news_articles_kn_result.csv', index=False)